### Расчеты параметров робота для эмуляции 


Расчет инерциальных параметров робота (Inertia tensor)

In [1]:
from sympy import sympify, Symbol, symbols, Matrix, init_printing, pretty
init_printing()

In [2]:
# символы/параметры описывающие модель
mass = symbols('m', real=True)
height = symbols('h', real=True)
width = symbols('w', real=True)
depth = symbols('d', real=True)
radius = symbols('r', real=True)

In [3]:
# Тензор инерции для объека кубической формы
ixx = sympify("1/12*m*(h**2 + d**2)")
iyy = sympify("1/12*m*(w**2 + h**2)")
izz = sympify("1/12*m*(w**2 + d**2)")
ixy = ixz = iyz = iyx = izy = izx = 0.0

CT = Matrix([
    [ixx, ixy, ixz], 
    [iyx, iyy, iyz], 
    [izx, izy, izz], 
])
CT

⎡  ⎛ 2    2⎞                          ⎤
⎢m⋅⎝d  + h ⎠                          ⎥
⎢───────────       0            0     ⎥
⎢     12                              ⎥
⎢                                     ⎥
⎢               ⎛ 2    2⎞             ⎥
⎢             m⋅⎝h  + w ⎠             ⎥
⎢     0       ───────────       0     ⎥
⎢                  12                 ⎥
⎢                                     ⎥
⎢                            ⎛ 2    2⎞⎥
⎢                          m⋅⎝d  + w ⎠⎥
⎢     0            0       ───────────⎥
⎣                               12    ⎦

In [6]:
# Расчет определителя по физическим данным модели 
mass = 50.0  # kg
height = 0.8 # meters
width = 0.8  # meters
depth = 0.8  # meters

values = { 'm': mass, 'h': height, 'w': width, 'd': depth }
ixx.subs(values)
iyy.subs(values)
izz.subs(values)
CTR = CT.subs(values)

print("Момент инерции (кубоида): ", CTR.det())

Момент инерции (кубоида):  151.703703703704


In [62]:
# Тензор инерции для объекта сферической формы
ixx = iyy = izz = sympify("2/5*m*r**2")
ixy = ixz = iyz = iyx = izy = izx = 0.0

ST = Matrix([
    [ixx, ixy, ixz], 
    [iyx, iyy, iyz], 
    [izx, izy, izz], 
])
ST

⎡     2                ⎤
⎢2⋅m⋅r                 ⎥
⎢──────    0       0   ⎥
⎢  5                   ⎥
⎢                      ⎥
⎢             2        ⎥
⎢        2⋅m⋅r         ⎥
⎢  0     ──────    0   ⎥
⎢          5           ⎥
⎢                      ⎥
⎢                     2⎥
⎢                2⋅m⋅r ⎥
⎢  0       0     ──────⎥
⎣                  5   ⎦

In [65]:
# Расчет определителя по физическим данным модели 
mass = 0.7   # kg
radius = 0.2 # meters

values = { 'm': mass, 'r': radius }
ixx.subs(values)
iyy.subs(values)
izz.subs(values)

STR = ST.subs(values)

print("Момент инерции (сфера): ", STR.det())

Момент инерции (сфера):  1.40492800000000e-6


In [1]:
[0.5]*3

[0.5, 0.5, 0.5]